In [4]:
#!{sys.executable} -m pip install matplotlib

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as widgets

In [92]:
def filter_df(df_orig, col, arg, op='=='):
    df = df_orig.copy()
    
    if op == '==':
        df = df.loc[df[col]==arg]
    elif op == '<':
        df = df.loc[df[col]<arg]
    elif op == '>':
        df = df.loc[df[col]>arg]
    elif op == '<=':
        df = df.loc[df[col]<=arg]
    elif op == '>=':
        df = df.loc[df[col]>=arg]
    elif op == '!=':
        df = df.loc[df[col]!=arg]
    else:
        print('op type not supported!')
    
    return df

In [96]:
config_df = pd.read_csv("../cmake-build-debug/config_log.txt", sep=",", header=0)
display(config_df)
config_dict = config_df.to_dict(orient='list')


sim_df = pd.read_csv("../cmake-build-debug/sim_log.txt", sep=",", header=0)
display(sim_df.head())

sampling_df = pd.read_csv("../cmake-build-debug/sampling_log.txt", sep=",", header=0)
display(sampling_df.head())

,sim_time,rollouts,horizon,obstacle_cost,obstacle_rad_0,obstacle_rad_1,obstacle_rad_2,obstacle_rad_3,obstacle_pos_0_x,obstacle_pos_0_y,obstacle_pos_1_x,obstacle_pos_1_y,obstacle_pos_2_x,obstacle_pos_2_y,obstacle_pos_3_x,obstacle_pos_3_y,target_state_0,target_state_1,Unnamed: 18
0,100,50,20,10000,10,5,10,5,5,20,55,15,40,20,80,40,100,50,NaN


,simstep,state_0,state_1,state_2,state_3,state_4,state_5,Unnamed: 7
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
1,1,0.450181,-0.431339,0.900362,-0.862678,0.900362,-0.862678,NaN
2,2,1.556290,-1.177430,1.311860,-0.629510,0.411503,0.233168,NaN
3,3,3.069110,-1.288970,1.713770,0.406441,0.401909,1.035950,NaN
4,4,4.860900,-0.435660,1.869810,1.300170,0.156033,0.893732,NaN


,simstep,step,rollout,state_0,state_1,state_2,state_3,state_4,state_5,cost_cum,Unnamed: 10
0,0,0,0,-0.028586,-0.133596,-0.057173,-0.267192,-0.057173,-0.267192,223.692,NaN
1,0,0,1,0.159660,-0.066349,0.319320,-0.132699,0.319320,-0.132699,223.494,NaN
2,0,0,2,0.036145,-0.228998,0.072291,-0.457995,0.072291,-0.457995,223.677,NaN
3,0,0,3,-0.045945,0.042808,-0.091891,0.085616,-0.091891,0.085616,223.629,NaN
4,0,0,4,-0.238331,0.107248,-0.476662,0.214496,-0.476662,0.214496,223.772,NaN


In [97]:
#df_timestep = filter_df(sampling_df, "simstep", 10)
#df_timestep = filter_df(df_timestep, "step", 3)

In [98]:
def draw_interactive(simstep, step):
    df_timestep = filter_df(sampling_df, "simstep", simstep, op='==')
    df_timestep = filter_df(df_timestep, "step", step, op='<=')
    
    fig, ax = fig, ax = plt.subplots(1, figsize=(15,10))
   
    if config_dict['obstacle_cost'][0] > 0:
        for obstacle_nr in range(0,4):
            obst_id_rad = 'obstacle_rad_'+str(obstacle_nr)
            obst_id_x = 'obstacle_pos_'+str(obstacle_nr)+'_x'
            obst_id_y = 'obstacle_pos_'+str(obstacle_nr)+'_y'
            
            ellipse_plot_0 = patches.Circle((config_dict[obst_id_x][0],config_dict[obst_id_y][0]), 
                                     radius=config_dict[obst_id_rad][0], color='gray')
      
            ax.add_patch(ellipse_plot_0)
    
    ax.plot(sim_df[['state_0']][:simstep+1], sim_df[['state_1']][:simstep+1], c='r')
    ax.plot(sim_df[['state_0']], sim_df[['state_1']], c='r', alpha=0.3)
    ax.scatter(df_timestep[['state_0']], df_timestep[['state_1']], s=10, c='g',zorder=10)
    ax.scatter(sim_df[['state_0']].iloc[simstep], sim_df[['state_1']].iloc[simstep],s=60, c='r',zorder=20)


    padding = 10;
    ax.set_xlim(sim_df[['state_0']].min()[0]-padding, sim_df[['state_0']].max()[0]+padding)
    ax.set_ylim(sim_df[['state_1']].min()[0]-padding, sim_df[['state_1']].max()[0]+padding)
    plt.show()
    
widgets.interactive(draw_interactive, 
                    simstep = widgets.IntSlider(value=0, min=0, max=config_dict['sim_time'][0]-1, step=1), 
                    step= widgets.IntSlider(value=19, min=0, max=config_dict['horizon'][0]-1))

interactive(children=(IntSlider(value=0, description='simstep', max=99), IntSlider(value=19, description='step…

In [11]:
sim_df[['state_0']].min()[0]

-3.01472